In [29]:
from collections import Counter
from pathlib import Path

import pandas as pd
import spacy
from cleantext import clean
from spacy_iwnlp import spaCyIWNLP

In [30]:
nlp = spacy.load("de_core_news_md")
iwnlp = spaCyIWNLP(lemmatizer_path='IWNLP.Lemmatizer_20181001.json')
nlp.add_pipe(iwnlp)

In [31]:
filenames = ! fd eu_ eu-wahlprogramme

In [32]:
filenames

['eu-wahlprogramme/eu_afd.md',
 'eu-wahlprogramme/eu_cdu.md',
 'eu-wahlprogramme/eu_dielinke.md',
 'eu-wahlprogramme/eu_fdp.md',
 'eu-wahlprogramme/eu_gruene.md',
 'eu-wahlprogramme/eu_spd.md']

In [33]:
data = [Path(fn).read_text() for fn in filenames]

In [34]:
data = [d.replace('#', '') for d in data]

In [35]:
data = [clean(d, lang='de', lower=False) for d in data]

In [36]:
data = [[t for t in list(nlp(d)) if t.pos_ in ['NOUN', 'PROPN'] and str(t) != '"'] for d in data]

In [37]:
data = [[str(t if t._.iwnlp_lemmas is None else t._.iwnlp_lemmas[0]) for t in d] for d in data]

In [38]:
data = [[t.replace('Deutschlands', 'Deutschland').replace('Europas', 'Europa').replace('Mitgliedsstaat', 'Mitgliedstaat') for t in d] for d in data]

In [39]:
cs = [Counter(d) for d in data]

In [40]:
def proper_ranking(c):
    res = {}
    prev_count = -1
    prev_rank = -1
    for rank, pair in enumerate(c.most_common()):
        if pair[1] != prev_count:
            new_rank = rank
        else:
            new_rank: prev_rank
        res[pair[0]] = new_rank
        prev_rank = new_rank
        prev_count = pair[1]
    return res

In [41]:
rankings = [proper_ranking(c) for c in cs]

In [42]:
rankings[0]

{'EU': 0,
 'AfD': 1,
 'Deutschland': 2,
 'Europa': 3,
 'Staat': 4,
 'Bürger': 5,
 'Land': 6,
 'Jahr': 6,
 'Euro': 8,
 'Mitgliedstaat': 9,
 'Mensch': 10,
 'Ziel': 11,
 'Recht': 12,
 'Kind': 12,
 'Nationalstaat': 14,
 'Gesellschaft': 15,
 'Entwicklung': 16,
 'Ebene': 16,
 'Familie': 16,
 'Kultur': 19,
 'Interesse': 19,
 'Unternehmen': 19,
 'EZB': 19,
 'Union': 23,
 'Freiheit': 23,
 'Bevölkerung': 23,
 'Bereich': 26,
 'Folge': 26,
 'Million': 26,
 'Hilfe': 26,
 'Sprache': 30,
 'Politik': 30,
 'Zusammenarbeit': 30,
 'Maßnahme': 30,
 'Bundesregierung': 30,
 'Wiedereinführung': 30,
 'Förderung': 30,
 'Verbraucher': 30,
 'Zeit': 38,
 'Vertrag': 38,
 'Entscheidung': 38,
 'Wirtschaft': 38,
 'Erhalt': 38,
 'Schutz': 38,
 'Landwirtschaft': 38,
 'Zuwanderung': 38,
 'Islam': 38,
 'Verbraucherschutz': 38,
 'Versuch': 48,
 'Identität': 48,
 'Haftung': 48,
 'Verbot': 48,
 'Kontrolle': 48,
 'Souveränität': 48,
 'Zahl': 48,
 'Aufgabe': 48,
 'Regel': 48,
 'Mittel': 48,
 'Entwicklungshilfe': 48,
 'Sicherh

In [43]:
cs[2].most_common(30)

[('EU', 254),
 ('Europa', 168),
 ('Mensch', 155),
 ('LINKE', 96),
 ('Recht', 83),
 ('Beschäftigte', 64),
 ('Land', 63),
 ('Deutschland', 59),
 ('Unternehmen', 56),
 ('Union', 49),
 ('Mitgliedstaat', 49),
 ('Arbeit', 48),
 ('Konzern', 44),
 ('Lohn', 42),
 ('Prozent', 39),
 ('Politik', 38),
 ('Bildung', 38),
 ('Demokratie', 33),
 ('Region', 33),
 ('Frau', 32),
 ('Armut', 32),
 ('Zugang', 32),
 ('Investition', 31),
 ('Jahr', 30),
 ('Profit', 29),
 ('Leben', 29),
 ('Million', 28),
 ('Interesse', 28),
 ('Parlament', 28),
 ('Förderung', 28)]

In [44]:
# choose the top 20 words for each party
s = set()
for r in rankings:
    for w, rank in r.items():
        if rank < 20:
            s.add(w)
        else:
            break
s

{'AfD',
 'Arbeit',
 'Arbeitsplatz',
 'Armut',
 'Beispiel',
 'Bereich',
 'Beschäftigte',
 'Bildung',
 'Bürger',
 'Bürgerin',
 'CDU',
 'Chance',
 'Demokrat',
 'Demokratie',
 'Deutschland',
 'EU',
 'EZB',
 'Ebene',
 'Entwicklung',
 'Euro',
 'Europa',
 'Familie',
 'Forschung',
 'Frau',
 'Freiheit',
 'Gesellschaft',
 'Innovation',
 'Interesse',
 'Jahr',
 'Kind',
 'Kommune',
 'Konzern',
 'Kultur',
 'LINKE',
 'Land',
 'Lohn',
 'Mensch',
 'Menschenrecht',
 'Mitgliedstaat',
 'Nationalstaat',
 'Parlament',
 'Politik',
 'Prozent',
 'Recht',
 'Region',
 'Schutz',
 'Sicherheit',
 'Staat',
 'Union',
 'Unternehmen',
 'Weg',
 'Welt',
 'Ziel',
 'Zugang',
 'Zukunft',
 'Zusammenarbeit'}

In [45]:
exclude = ['AfD', 'CDU', 'LINKE', 'SPD']

In [46]:
parties = ['AfD', 'CDU', 'Linke', 'FDP', 'Grüne', 'SPD']

In [47]:
final_words = [x for x in s if x not in exclude]

In [48]:
rows = []
for w in final_words:
    for p_idx, r in enumerate(rankings):
        value = -1
        if w in r:
            value = r[w]
        rows.append({'value': value, 'word': w, 'party': parties[p_idx]})
df = pd.DataFrame(rows)

In [49]:
df.to_csv('data.csv', index=False)

In [50]:
df

,party,value,word
0,AfD,2,Deutschland
1,CDU,2,Deutschland
2,Linke,7,Deutschland
3,FDP,31,Deutschland
4,Grüne,16,Deutschland
5,SPD,21,Deutschland
6,AfD,16,Entwicklung
7,CDU,17,Entwicklung
8,Linke,32,Entwicklung
9,FDP,23,Entwicklung


In [51]:
df_comp = df
df_comp['value'] = df_comp['value'].apply(lambda x: 2000 if x == -1 else x)

In [52]:
df_comp

,party,value,word
0,AfD,2,Deutschland
1,CDU,2,Deutschland
2,Linke,7,Deutschland
3,FDP,31,Deutschland
4,Grüne,16,Deutschland
5,SPD,21,Deutschland
6,AfD,16,Entwicklung
7,CDU,17,Entwicklung
8,Linke,32,Entwicklung
9,FDP,23,Entwicklung


In [53]:
w_c = [[df_comp[df_comp['word'] == w]['value'].sum(), w] for w in final_words]

In [54]:
'", "'.join([x[1] for x in sorted(w_c, key=lambda x: x[0], reverse=True)])

'Demokrat", "Nationalstaat", "Bürgerin", "EZB", "Lohn", "Beschäftigte", "Kommune", "Chance", "Familie", "Armut", "Konzern", "Bürger", "Innovation", "Prozent", "Beispiel", "Kind", "Weg", "Arbeitsplatz", "Arbeit", "Frau", "Region", "Welt", "Forschung", "Kultur", "Gesellschaft", "Zugang", "Demokratie", "Zusammenarbeit", "Euro", "Menschenrecht", "Ebene", "Bildung", "Politik", "Bereich", "Sicherheit", "Zukunft", "Freiheit", "Interesse", "Parlament", "Schutz", "Jahr", "Staat", "Entwicklung", "Ziel", "Deutschland", "Land", "Union", "Recht", "Unternehmen", "Mitgliedstaat", "EU", "Mensch", "Europa'